In [1]:
from qcc.ml import Module

class Test(Module):
    @Module.parameter(1)
    def a(self):
        return 1
    
    @Module.parameter(0)
    def b(self):
        return 2
    
t = Test()

for x in t.parameters():
    print(x)

Parameter containing:
tensor([-1.6528,  5.6806], requires_grad=True)
Parameter containing:
tensor([3.7175], requires_grad=True)
